From explore.ipynb we can find the images that we can augment by rotating, flipping or mirroring.
## TODO

Ideas:

- Balance data by augmentation (Create more new pictures for underrepresented signs)
- Assign different signs to different augmentation methods --> What is possible for which method?

Signs:

- 1) Speed Limit Zone beginning (30) --> Ordner: 00001
- 2) Speed Limit Zone end (30) --> Ordner: KEINE DATEN bis jetzt
- 3) Crosswalk --> Ordner: KEINE DATEN bis jetzt
- 4) Barred area --> Ordner: KEINE DATEN bis jetzt
- 5) Expressway beginning --> Ordner: KEINE DATEN bis jetzt
- 6) Expressway end --> Ordner: KEINE DATEN bis jetzt
- 7) Intersections STOP --> Ordner: 00014
- 8) Intersections "Vorfahrt gewähren" --> Ordner: 00013
- 9) Intersections "Vorfahrtsstraße"  --> Ordner: 00012
- 10) Turn Left  --> Ordner: 00034
- 11) Turn Right --> Do we need the counterparts here?  --> Ordner: 00033
- 12) No-passing zone beginning --> Ordner: 00009
- 13) No-passing zone end  --> Ordner: 00041
- 14) Sharp turn left small --> Ordner: KEINE DATEN bis jetzt
- 15) Sharp turn left large --> Ordner: KEINE DATEN bis jetzt
- 16) Sharp turn right small --> Do we need the counterparts here? --> Ordner: KEINE DATEN bis jetzt
- 17) Sharp turn right large --> Do we need the counterparts here? --> Ordner: KEINE DATEN bis jetzt
- 18) Steep hill uphill grade --> Ordner: KEINE DATEN bis jetzt
- 19) Steep hill downhill grade --> Ordner: KEINE DATEN bis jetzt

Methods to augment data + for which signs applicable:

-Flipping (horizontal/vertical)
    
    --> check possible invariances (e.g turn left signs) 
        and signs which can then be detected as other signs
        
    Horizontal:
    9)
    
    Vertical:
    5), 8), 9), 10) gets 11), 11) gets 10), 16) gets 17), 17) gets 16)
    
    Horizontal + Vertical:
    9)

-Rotating




## TODO

In [ ]:
fippable_horizontally = [1, 5]

In [ ]:
#Dict list for signs ('name': 'folderdirectory')
dict1 = {'Speed Limit Zone beginning (30)': '/00001', 'Speed Limit Zone end (30)': '-', 'Crosswalk': '-', 'Barred area': '-', 'Expressway beginning': '-', 'Expressway end': '-', 'Intersections STOP': '/00014', 'Intersections "Vorfahrt gewähren"': '/00013', 'Intersections "Vorfahrtsstraße"': '/00012', 'Turn Left': '/00034', 'Turn Right': '/00033', 'No-passing zone beginning': '/00009', 'No-passing zone end': '/00041', 'Sharp turn left small': '-', 'Sharp turn left large': '-', 'Sharp turn right small': '-', 'Sharp turn right large': '-', 'Steep hill uphill grade': '-', 'Steep hill downhill grade': '-'}

#Print all sign types + link to the folders
for el in dict1:
    print (el, "-->", dict1[el], )

In [ ]:
#Count images per cell
from gtsrb_loader.get_folderpath import get_folderpath
import os, os.path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

path=get_folderpath('train');

number_list = [] #element 0 is 0
sign_list = []

i = 0
for k in iter(dict1.keys()):
    if dict1[k] == '-':
        number_files = 0
    else:
        changed_path=path+dict1[k]
        list = os.listdir(changed_path) # dir is your directory path
        number_files = len(list)
    number_list.append(number_files)
    sign_list.append(k)

#Histogram
plt.bar(range(len(number_list)), number_list)
plt.xticks(range(1, len(number_list)))
plt.title("Number of Files per folder")
plt.xlabel("Signs")
plt.ylabel("Number of Files")
plt.xticks(range(len(number_list)), sign_list, rotation='vertical')
plt.show()

#Legende
for i in range(1, len(number_list)):
    print (str(i) + ": " + sign_list[i] + " --> " + str(number_list[i]))

## Augment cells using Keras 

Basically, should do what is done here for our masks: https://keras.io/preprocessing/image/ <br/>
So first, we create mask images for each of our size-equalized images:

In [ ]:
from gtsrb_loader.load_data import load_data
import numpy as np

# load the images and classes
X_train, y_train = load_data(path)
X_train, y_train = np.array(X_train), np.array(y_train).astype(int)

In [ ]:
import pandas as pd
import os

images_df = pd.DataFrame()

# load all csv label files, and save new image path to faciliate saving of the images in the next step
for root, dirs_list, files_list in os.walk(path):
    for file_name in files_list:
        if file_name.endswith(".csv"):
            file_name_path = os.path.join(root, file_name)
            current_csv = pd.read_csv(file_name_path, sep=';')
            for i,row in current_csv.iterrows():
                new_name = os.path.join(os.path.dirname(os.path.dirname(root)),'Masks',os.path.basename(os.path.normpath(root)), current_csv.get_value(i,'Filename'))
                current_csv.set_value(i,'Filename',new_name)
            images_df = images_df.append(current_csv,ignore_index=True)

Now we have to generate mask images (255 at ROI, 0 at rest)

In [ ]:
from PIL import Image, ImageDraw

# top-level folder
top_level_folder = os.path.dirname(os.path.dirname(images_df.get_value(1,'Filename')))
if not os.path.isdir(top_level_folder):
    os.mkdir(top_level_folder)
    print('Created top-level folder', top_level_folder)

# now: generate images and transform them
for i,row in images_df.iterrows():
    width = images_df.get_value(i,'Width')
    height = images_df.get_value(i,'Height')
    roi_x1 = images_df.get_value(i, 'Roi.X1')
    roi_y1 = images_df.get_value(i, 'Roi.Y1')
    roi_x2 = images_df.get_value(i, 'Roi.X2')
    roi_y2 = images_df.get_value(i, 'Roi.Y2')
    
    im = Image.new('1', (width,height), (0))
    draw = ImageDraw.Draw(im)
    draw.rectangle([(roi_x1, roi_y1), (roi_x2, roi_y2)],fill=255)
    
    # make subfolders for each sign class if necessary
    if not os.path.isdir(os.path.dirname(images_df.get_value(i,'Filename'))):
        os.mkdir(os.path.dirname(images_df.get_value(i,'Filename')))
    
    # save created image at previously created path
    im.save(images_df.get_value(i,'Filename'))

Now, we load both the images and the corresponding masks:

In [ ]:
from scipy import ndimage

# load masks as numpy arrays
shape_X = (X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
shape_Y = y_train.shape
Mask_X_train = np.empty(shape_X)
Mask_Y_train = np.empty(shape_Y)
for i,row in images_df.iterrows():
    Mask_X_train[i] = ndimage.imread(images_df.get_value(i,'Filename'),flatten=True).reshape(shape_X[1], shape_X[2], 1)
    Mask_Y_train[i] = images_df.get_value(i,'ClassId')

Finally, we augment using Keras data generators:

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# we create two instances with the same arguments 
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)
print('Created image data generators')
    
# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
print('Fitting data generators')
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(Mask_X_train, augment=True, seed=seed)

In [ ]:
# rename our previously defined paths for clarity
image_path = path
mask_path = top_level_folder

image_path_out = os.path.join(os.path.dirname(image_path),'Images_Augmented')
if not os.path.isdir(image_path_out):
    os.mkdir(image_path_out)
    print('Created folder for augmented images at:', image_path_out)
else:
    print('Using existing augmented image folder at:', image_path_out)

if not os.path.isdir(os.path.join(image_path_out,'Images')):
    os.mkdir(os.path.join(image_path_out,'Images'))
if not os.path.isdir(os.path.join(image_path_out,'Masks')):
    os.mkdir(os.path.join(image_path_out,'Masks'))  

# set this to specify how many images you want to process at once
batching_size = 32;   
# set this to specify how many times you want to process that many images
repeats = 1

# flow and directly augment to folders
i = 0
for batch in image_datagen.flow(X_train, batch_size=batching_size,
                          save_to_dir=os.path.join(image_path_out,'Images'), save_prefix='mod', save_format='ppm'):
    i += 1
    if i > repeats:
        break

# augment masks
i = 0
for batch in mask_datagen.flow(Mask_X_train, batch_size=batching_size,
                          save_to_dir=os.path.join(image_path_out,'Masks'), save_prefix='mod', save_format='ppm'):
    i += 1
    if i > repeats:
        break

This is how you would then train a model with this data

In [ ]:
from keras.models import Model

# combine generators into one which yields image and masks
image_generator = image_datagen.flow(X_train)
mask_generator = mask_datagen.flow(Mask_X_train)
train_generator = zip(image_generator, mask_generator)

model.fit_generator(
    train_generator,
    steps_per_epoch=2000,
    epochs=50)